In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
import shapely

In [ ]:
# rm(list=ls()) 

# library(devtools)


# library(choroplethrZip)
# library(tidyverse)

# https://github.com/arilamstein/choroplethrZip

In [ ]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [ ]:
df.head()

In [ ]:
df['reportedZipCode'].dtypes

In [ ]:
df['reportedZipCode'].isna().sum() / df.shape[0]

In [ ]:
df['censusBlockGroupFips'].isna().sum() / df.shape[0]

In [ ]:
df['latitude'].isna().sum() / df.shape[0]

In [ ]:
df['longitude'].isna().sum() / df.shape[0]

In [ ]:
missing_all_three = df['reportedZipCode'].isna() | df['censusBlockGroupFips'].isna() | df['latitude'].isna()

# To get the number of such rows
num_missing_all_three = missing_all_three.sum()

# To get the proportion of such rows
proportion_missing_all_three = num_missing_all_three / df.shape[0]

In [ ]:
proportion_missing_all_three*100

In [ ]:
#Zipcode list

list_zipcode = df['reportedZipCode'].dropna().drop_duplicates().astype(int).astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode = [zipcode.zfill(5) for zipcode in list_zipcode]

# temp = pd.DataFrame(list_censusBG, columns=['ZipCode'])

# temp.to_csv('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/ZipCode.csv', index=False)

In [ ]:
#CensusBG list

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

# temp = pd.DataFrame(list_censusBG, columns=['censusBG'])

# temp.to_csv('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/censusBG.csv', index=False)

In [ ]:
#lat long df

df_lat_long = df[['latitude', 'longitude']].dropna().drop_duplicates()

## Using pygris

In [ ]:
years = [2000, 2010, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [ ]:
# zip_codes_list = pygris.zctas(year=2000)

In [ ]:
# zip_codes_list.head()

In [ ]:
# zip_codes_list.shape[0]

In [ ]:
# years = [2000, 2010, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

years = [2000, 2010, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

yearly_data = {}

for i in years:
    yearly_data[i] = pygris.zctas(year=i)

In [ ]:
# temp = pygris.zctas(year=2000)

In [ ]:
temp = pd.DataFrame(yearly_data[2021])

In [ ]:
temp.head()

In [ ]:
temp = temp[['ZCTA5CE20', 'geometry']]

In [ ]:
temp = temp.rename(columns={"ZCTA5CE20": "ZIPcode"})
temp['year'] = 2021
temp

In [ ]:
list_zipcode_temp = temp['ZIPcode'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode_temp = [zipcode.zfill(5) for zipcode in list_zipcode_temp]

In [ ]:
len(list_zipcode_temp)

In [ ]:
list_zipcode_3 = list_zipcode_3 + list_zipcode_temp

In [ ]:
len(list_zipcode_3)

In [ ]:
list_zipcode_3 = list(set(list_zipcode_3))

In [ ]:
len(list_zipcode_3)

In [ ]:
set_zipcode = set(list_zipcode)
set_zipcode_3 = set(list_zipcode_3)

# Find the intersection of the two sets
common_zipcodes = set_zipcode.intersection(set_zipcode_3)

# Print the number of common zip codes
print("Number of common zip codes:", len(common_zipcodes))

In [ ]:
len(common_zipcodes)/len(list_zipcode)

In [ ]:
zip_code_df2000 = temp[temp['ZIPcode'].isin(common_zipcodes)]

In [ ]:
zip_code_df2000.head()

In [ ]:
zip_code_df2010 = temp[temp['ZIPcode'].isin(common_zipcodes)]

In [ ]:
zip_code_df2010.head()

In [ ]:
zip_code_df2012 = temp[temp['ZIPcode'].isin(common_zipcodes)]

In [ ]:
zip_code_df2012.head()

In [ ]:
zip_code_df2020 = temp[temp['ZIPcode'].isin(common_zipcodes)]

In [ ]:
zip_code_df2020.head()

In [ ]:
zip_code_df2021 = temp[temp['ZIPcode'].isin(common_zipcodes)]

In [ ]:
zip_code_df2021.head()

In [ ]:
zip_code_df = pd.concat([zip_code_df2000, zip_code_df2010, zip_code_df2012, zip_code_df2020, zip_code_df2021], ignore_index=True)

In [ ]:
zip_code_df = zip_code_df.drop_duplicates(subset=['ZIPcode', 'geometry'])

In [ ]:
zip_code_df.shape[0]

In [ ]:
zip_code_df.head()

In [ ]:
# Convert the geometry column to WKT format
zip_code_df['geometry'] = zip_code_df['geometry'].apply(lambda geom: geom.wkt)

In [ ]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(zip_code_df), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = zip_code_df.iloc[start:end].copy()
    # temp_df['geometry'] = temp_df['geometry'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip", compression='gzip')

In [2]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_0_25000.parquet.gzip")

# Convert the WKT strings back to geometries
gdf_read = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [3]:
df_read.head()

,ZIPcode,geometry,year
0,38739,"MULTIPOLYGON (((-90.499253 34.053217, -90.4997...",2000
1,38767,"POLYGON ((-90.76477 34.119506, -90.766978 34.1...",2000
2,38669,"POLYGON ((-90.74080599999999 34.184565, -90.74...",2000
3,38630,"MULTIPOLYGON (((-90.67204699999999 34.259564, ...",2000
4,38639,"POLYGON ((-90.456458 34.31975, -90.456758 34.3...",2000


In [5]:
import geopandas as gpd
from shapely.wkt import loads

# Assuming you have a DataFrame df with columns 'unit' and 'geometry'
# Example:
#    unit  |  geometry
#   -------|-----------
#    unit1 |  geom1
#    unit2 |  geom2
#    unit3 |  geom3

# Here's a hypothetical dataset for demonstration
data = {
    'unit': ['unit1', 'unit2', 'unit3'],
    'geometry': ['POLYGON ((0 0, 0 2, 2 2, 2 0, 0 0))',
                'POLYGON ((1 1, 1 3, 3 3, 3 1, 1 1))',
                'POLYGON ((0.5 0.5, 0.5 2.5, 2.5 2.5, 2.5 0.5, 0.5 0.5))']
}

# Convert string representations of geometries to actual Shapely geometry objects
data['geometry'] = [loads(geom) for geom in data['geometry']]

df = gpd.GeoDataFrame(data, geometry='geometry')

# Get the intersection of the three geometries
intersection_geom = df['geometry'].iloc[0]
for geom in df['geometry'].iloc[1:]:
    intersection_geom = intersection_geom.intersection(geom)

print(intersection_geom)

POLYGON ((2 1, 1 1, 1 2, 2 2, 2 1))


In [ ]:
# Drop duplicates
zip_code_df = zip_code_df.drop_duplicates(subset=['ZIPcode'])

In [ ]:
list_zipcode_2 = zip_codes_list['ZCTA5CE20'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode_2 = [zipcode.zfill(5) for zipcode in list_zipcode_2]

In [ ]:
# Convert both lists to sets
set_zipcode = set(list_zipcode)
set_zipcode_2 = set(list_zipcode_2)

# Find the intersection of the two sets
common_zipcodes = set_zipcode.intersection(set_zipcode_2)

# Print the number of common zip codes
print("Number of common zip codes:", len(common_zipcodes))

In [ ]:
len(common_zipcodes)/len(list_zipcode)

In [ ]:
states = pygris.states()
states_list = sorted(states['STUSPS'].unique())

In [ ]:
zip_codes_list['ZCTA5CE20'] = zip_codes_list['ZCTA5CE20'].astype(str)

filtered_df = zip_codes_list[zip_codes_list['ZCTA5CE20'].isin(list_zipcode)]

In [ ]:
filtered_df.head()

In [ ]:
years2 = [1990, 2000, 2010, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [ ]:
BG_list = []

for i in states_list:
    BG_list.append(pygris.block_groups(state = i, year = 2021))

In [ ]:
BG_list = []

for i in states_list:
    BG_list.append(pygris.block_groups(state = i, year = 2010))

In [ ]:
BG_list = []

for i in states_list:
    BG_list.append(pygris.block_groups(state = i, year = 2000))

In [ ]:
BG_list = []

for i in states_list:
    BG_list.append(pygris.block_groups(state = i, year = 1990))

In [ ]:
BG_df = pd.concat(BG_list, ignore_index=True)

In [ ]:
BG_df.head()

In [ ]:
list_censusBG_2 = BG_df['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG_2 = [censusBG.zfill(12) for censusBG in list_censusBG_2]

In [ ]:
len(list_censusBG_2)

In [ ]:
list_censusBG_3 = list_censusBG_3 + list_censusBG_2 

In [ ]:
len(list_censusBG_3)

In [ ]:
list_censusBG_3 = list(set(list_censusBG_3))

In [ ]:
len(list_censusBG_3)

In [ ]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [ ]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_3)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

In [ ]:
len(common_bg)/len(list_censusBG)

In [ ]:
len(common_bg)

In [ ]:
len(list_censusBG)

## Using Shapefiles

In [ ]:
# Read the shapefile
shapefile_path = 'C:/Users/Asus/Box/Flood Damage PredictionProject/Shapefiles/tl_rd22_us_zcta520/tl_rd22_us_zcta520.shp'  # Replace with the path to your shapefile
map_df = gpd.read_file(shapefile_path)

In [ ]:
map_df.head()

In [ ]:
list_zipcode_2 = map_df['ZCTA5CE20'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode_2 = [zipcode.zfill(5) for zipcode in list_zipcode]

In [ ]:
# Convert both lists to sets
set_zipcode = set(list_zipcode)
set_zipcode_2 = set(list_zipcode_2)

# Find the intersection of the two sets
common_zipcodes = set_zipcode.intersection(set_zipcode_2)

# Print the number of common zip codes
print("Number of common zip codes:", len(common_zipcodes))

In [ ]:
len(common_zipcodes)/len(list_zipcode)